In [1]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 719.8/719.8 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 77.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 49.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 54.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 126.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 89.1 MB/s eta 0:00:00
Mounted at /content/gdrive


In [2]:
#hide
from fastbook import *
from IPython.display import display,HTML

fastai's Layered API

In [3]:
from fastai.text.all import *

dls = TextDataLoaders.from_folder(untar_data(URLs.IMDB), valid='test')

In [4]:
path = untar_data(URLs.IMDB)
dls = DataBlock(
    blocks=(TextBlock.from_folder(path),CategoryBlock),
    get_y = parent_label,
    get_items=partial(get_text_files, folders=['train', 'test']),
    splitter=GrandparentSplitter(valid_name='test')
).dataloaders(path)

Transforms

In [5]:
files = get_text_files(path, folders = ['train', 'test'])
txts = L(o.open().read() for o in files[:2000])

In [6]:
tok = Tokenizer.from_folder(path)
tok.setup(txts)
toks = txts.map(tok)
toks[0]

(#198) ['xxbos','i','raced','to','the','library','to','check','out','this'...]

In [7]:
num = Numericalize()
num.setup(toks)
nums = toks.map(num)
nums[0][:10]

TensorText([   2,   19,    0,   15,    9, 3870,   15, 1059,   61,   20])

In [8]:
nums_dec = num.decode(nums[0][:10]); nums_dec

(#10) ['xxbos','i','xxunk','to','the','library','to','check','out','this']

In [9]:
tok.decode(nums_dec)

'xxbos i xxunk to the library to check out this'

In [10]:
tok((txts[0], txts[1]))

((#198) ['xxbos','i','raced','to','the','library','to','check','out','this'...],
 (#57) ['xxbos','i','have','seen','this','film','at','least','100','times'...])

Writing a transform

In [11]:
def f(x:int): return x+1
tfm = Transform(f)
tfm(2),tfm(2.0)

(3, 2.0)

In [12]:
@Transform
def f(x:int): return x+1
f(2),f(2.0)

(3, 2.0)

In [13]:
class NormalizeMean(Transform):
    def setups(self, items): self.mean = sum(items)/len(items)
    def encodes(self, x): return x-self.mean
    def decodes(self, x): return x+self.mean

In [14]:
tfm = NormalizeMean()
tfm.setup([1,2,3,4,5])
start = 2
y = tfm(start)
z = tfm.decode(y)
tfm.mean,y,z

(3.0, -1.0, 2.0)

Pipeline

In [15]:
tfms = Pipeline([tok, num])
t = tfms(txts[0]); t[:20]

TensorText([   2,   19,    0,   15,    9, 3870,   15, 1059,   61,   20, 4674,  120,  302,   68, 2241, 2469,   15,    9, 3114,   23])

In [16]:
tfms.decode(t)[:100]

'xxbos i xxunk to the library to check out this miniseries after having just finished listening to th'

TfmdLists and Datasets: Transformed Collections

TfmdLists

In [17]:
tls = TfmdLists(files, [Tokenizer.from_folder(path), Numericalize])

In [18]:
t = tls[0]; t[:20]

TensorText([    2,    19, 28220,    15,     9,  3462,    15,   841,    61,    20,  4954,   120,   282,    57,  1825,  2889,    15,     9,  3068,    22])

In [19]:
tls.decode(t)[:100]

'xxbos i raced to the library to check out this miniseries after having just finished listening to th'

In [20]:
tls.show(t)

xxbos i raced to the library to check out this miniseries after having just finished listening to the marvelous " talking books " unabridged version of the book . xxmaj the first half of this xxup tv version is really very good , but it stumbles quite a bit in the second half . xxmaj the relationship with the trustee is overplayed and conflicts are inserted between xxmaj jean and xxmaj joe that do n't exist in xxmaj shute 's story , unwisely in my opinion , as they greatly diminish the power of their love story . i was disappointed to find that the wonderful xxmaj bryan xxmaj brown 's xxmaj joe seemed a lot xxunk and much less appealing than the man in the book , but xxmaj helen xxmaj morse 's xxmaj jean was really quite good . i think they would have had to make this a 10 - hour miniseries to develop the outback story properly . xxmaj but all that said , i did watch whole thing in more or less one go and did appreciate its merits , all the while wishing that someone would do a less so

In [21]:
cut = int(len(files)*0.8)
splits = [list(range(cut)), list(range(cut,len(files)))]
tls = TfmdLists(files, [Tokenizer.from_folder(path), Numericalize],
                splits=splits)

In [22]:
tls.valid[0][:20]

TensorText([    2,     8,   453,    19,   217,   308,    16,   185,    15,    59,  1494,    15,     7, 12624,    10,     8,  5435,    23,   393,    11])

In [23]:
lbls = files.map(parent_label)
lbls

(#50000) ['pos','pos','pos','pos','pos','pos','pos','pos','pos','pos'...]

In [24]:
cat = Categorize()
cat.setup(lbls)
cat.vocab, cat(lbls[0])

(['neg', 'pos'], TensorCategory(1))